<a href="https://colab.research.google.com/github/allanpichardo/vae_synth/blob/main/Audio_Generation_with_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio Synthesis with Machine Learning

<a target="_blank" href="https://github.com/allanpichardo/vae_synth"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /><br>View source on GitHub</a>

<p>In this workshop, we will cover the basic intuition behind the variational autoencoder–a basic generative model. We will use a variety of synth samples as training data and use the final trained decoder to output a range of random wav files of unique sounds that can be used in a sampler or DAW.

# Overview

<ul>
  <li>Variational Autoencoders: Basic Intuition</li>
</ul>

In [ ]:
!pip install kapre==0.3.4

In [ ]:
!gdown --id 1dHQqjYClND3fLHjM--cOWQh_ucS480QI
!mkdir /content/samples
!unzip /content/samples.zip -d /content

In [ ]:
!git clone https://github.com/allanpichardo/vae_synth.git

In [ ]:
import os
from datetime import datetime
import tensorflow as tf
import librosa

In [6]:
import sys
sys.path.append('/content/vae_synth')

from vae_synth.models import get_model, get_synth_model
from vae_synth.callbacks import SpectrogramCallback
from vae_synth.generators import SoundSequence

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
path = '/content/samples'
sr = 44100
duration = 1.0
batch_size = 4
spectrogram_shape = (80, 1025)

In [ ]:
autoencoder = get_model(latent_dim=8, sr=sr, duration=duration)
autoencoder.stft.summary()
autoencoder.encoder.summary()
autoencoder.decoder.summary()

autoencoder.compile(optimizer=tf.keras.optimizers.Adam())

Model: "stft"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 132300, 1)]       0         
_________________________________________________________________
stft_mag_phase (Functional)  (None, 513, 513, 2)       0         
_________________________________________________________________
tf.image.resize_with_crop_or (None, 513, 513, 2)       0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 513, 513, 2) 0                                            
____________________________________________________________________________________

In [ ]:
sequence = SoundSequence(path, sr=sr, duration=duration, batch_size=batch_size)

In [ ]:
epochs = 20
autoencoder.fit(sequence, epochs=epochs)

Epoch 1/20
583/583 [==============================] - 43s 71ms/step - loss: 0.9214 - spectral_conv_loss: 0.9205 - kl_loss: 9.4986e-04
Epoch 2/20
583/583 [==============================] - 42s 72ms/step - loss: 0.9081 - spectral_conv_loss: 0.9071 - kl_loss: 9.0593e-04
Epoch 3/20
583/583 [==============================] - 41s 71ms/step - loss: 0.9104 - spectral_conv_loss: 0.9094 - kl_loss: 9.6872e-04
Epoch 4/20
583/583 [==============================] - 42s 71ms/step - loss: 0.9077 - spectral_conv_loss: 0.9073 - kl_loss: 3.8822e-04
Epoch 5/20
583/583 [==============================] - 41s 71ms/step - loss: 0.9067 - spectral_conv_loss: 0.9063 - kl_loss: 3.9430e-04
Epoch 6/20
583/583 [==============================] - 42s 71ms/step - loss: 0.9075 - spectral_conv_loss: 0.9070 - kl_loss: 4.6480e-04
Epoch 7/20
583/583 [==============================] - 41s 71ms/step - loss: 0.9081 - spectral_conv_loss: 0.9078 - kl_loss: 2.8784e-04
Epoch 8/20
583/583 [==============================] - 41s 71ms

In [ ]:
synth = get_synth_model(autoencoder.decoder)

In [ ]:
import librosa 

random = tf.random.normal([1, 8], 0, 1)
print(random)
wav = synth.predict_on_batch(random)
wav = librosa.util.normalize(wav[0])

tf.io.write_file('output.wav', tf.audio.encode_wav(wav, 44100))

from IPython.display import Audio
Audio(filename='output.wav', rate=44100)

tf.Tensor(
[[-0.22707355  1.5003572  -0.15913974  0.71210206  0.49416935  0.93992066
   1.4020661   0.3085858 ]], shape=(1, 8), dtype=float32)
